<a href="https://colab.research.google.com/github/RobbieJVMW/ft-llm/blob/main/Notebook_1_Before_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Generating a text message conversation before Fine Tuning
To check everything's set up OK.
Use a V100 runtime.

In [ ]:
# Install libraries

!pip install transformers accelerate sentencepiece bitsandbytes

In [ ]:
# On the Hugging Face site, Profile >> Settings >> Access Tokens (if you haven't already).
# Keep the token somewhere safe for future use.
# Paste that in to the field after running this cell. No need to "Add token as git credential"

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load the model

import torch
import transformers
from transformers import AutoModelForCausalLM,AutoTokenizer, BitsAndBytesConfig, pipeline

base_model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto",
)

In [ ]:
# Ask the model to continue a basic Text Message exchange

query = "<s><<SYS>>Write a realistic text message chat. Avoid repetition.<</SYS>>\n"
query += "[INST]Write a chat between John and Jane[/INST]\n"
query += "### John: Hi, Jane. How are you?\n"
query += "### Jane: "
text_gen = transformers.pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=300)
output = text_gen(query)
print(output[0]['generated_text'])